# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
vaca_df = pd.read_csv("output_data/cities.csv")
vaca_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
data = vaca_df.loc[:, ["latitude", "longitude"]]

dimensions = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

figure = gmaps.figure(layout=dimensions)
heatmap = gmaps.heatmap_layer(data, weights=vaca_df.humidity)
figure.add_layer(heatmap)
figure

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
mask = (vaca_df.wind_speed < 10) & (vaca_df.temperature < 80) & (vaca_df.temperature > 70) & (vaca_df.weather == "Clear")
preferred_df = vaca_df.loc[mask].reset_index(drop=True)
preferred_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = preferred_df
hotel_df.head()

In [ ]:
hotels = []

for indx, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        hotel = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    hotel = {"Hotel Name" : hotel,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    hotels.append(hotel)

In [ ]:
df = pd.DataFrame(hotels)
df.head()

In [ ]:
hotel_full = pd.merge(hotel_df, df, left_index=True, right_index=True)
hotel_full

In [ ]:
# Borrowed this code cell with some minor tweaks from the course instructor. Could not figure out how to get the dynammic
# "Hotel Name" to be included in the pop-ups.

infos = []

for indx, row in hotel_full.iterrows():
    info_box_template = f"""<dl>
                        <dt>Hotel Name:</dt><dd>{'Hotel Name'}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

In [ ]:
# Also Borrowed from the course instructor with some minor tweaks. 

# get data
data = vaca_df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_full.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=vaca_df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(hotel_data, info_box_content=infos)
fig.add_layer(markers)

# show the plot
fig

In [ ]:
# Could not understand how to make this work as it is written without changes, as the code notes say.

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_full.iterrows()]
locations = hotel_full[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
